In [2]:
from cosmoNODE.loaders import Anode as A
from torch.utils.data import DataLoader

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [9]:
from cosmoNODE.loaders import Big

In [58]:
adjoint = False

viz = True
viz_at_end = True

test_freq = 5

In [59]:
if adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint

In [54]:
class ODEFunc(nn.Module):

    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(1, 50),
            nn.Tanh(),
            nn.Linear(50, 1),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y)

In [15]:
def ode_batch(time_sols, flux_sols):
    s = torch.from_numpy(np.random.choice(np.arange(data_size - batch_time, dtype=np.int64), batch_size, replace=True))
    batch_y0 = flux_sols[s]  # (M, D)
    batch_t = time_sols[:batch_time]  # (T)
    batch_y = torch.stack([flux_sols[s + i] for i in range(batch_time)], dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y

In [11]:
big = Big()

In [12]:
import math

In [38]:
data_size = len(big.times)
batch_time = round(math.log2(data_size))
batch_size = data_size // 1000
print(f'data_size: {data_size}, batch_time: {batch_time}, batch_size: {batch_size}')

data_size: 1421705, batch_time: 20, batch_size: 1421


In [63]:
tmp_times = big.times.values.flatten()
data = big.data.values
data_dim = len(data[0, :])

In [77]:
times = torch.tensor(tmp_times).sort().values
tdata = torch.tensor(data)

data_0 = tdata[0].reshape(1, -1) # torch.Size([1, 1])
reshaped_data = tdata.reshape(-1, data_dim, 1)
data_0

times

tensor([59580.0343, 59580.0347, 59580.0347,  ..., 60674.3625, 60674.3625,
        60674.3630], dtype=torch.float64)

In [78]:
batch = ode_batch(ttimes, reshaped_data)

In [83]:
by0_f, bt_f, by_f = batch
print(f'by0_f.shape : {by0_f.shape}, bt_f.shape: {bt_f.shape}, by_f.shape: {by_f.shape}')
bt_f


by0_f.shape : torch.Size([1421, 16, 1]), bt_f.shape: torch.Size([20]), by_f.shape: torch.Size([20, 1421, 16, 1])


tensor([59580.0343, 59580.0347, 59580.0347, 59580.0347, 59580.0347, 59580.0347,
        59580.0352, 59580.0352, 59580.0352, 59580.0356, 59580.0356, 59580.0356,
        59580.0361, 59580.0361, 59580.0363, 59580.0363, 59580.0365, 59580.0365,
        59580.0365, 59580.0370], dtype=torch.float64)

In [80]:
epochs = 5
niters = 100
odefunc = ODEFunc()
optimizer = optim.RMSprop(odefunc.parameters(), lr=1e-3)
ii = 0
losses = []

In [81]:
r_tol = 1
a_tol = 1

In [82]:
for epoch in range(1, epochs+1):    
    for itr in range(1, niters + 1):
        optimizer.zero_grad()
        by0_f, bt_f, by_f = ode_batch(ttimes, reshaped_data)
        pred_f = odeint(odefunc, by0_f, bt_f, rtol=r_tol, atol=a_tol)
        loss = torch.mean(torch.abs(pred_f - by_f))
        loss.backward()
        optimizer.step()
#         print(itr)
        if itr % test_freq == 0:
            with torch.no_grad():
#                 clear_output()
                pred_interpolation = odeint(odefunc, flux_0, eval_times, rtol=r_tol, atol=a_tol)
                pred_f = odeint(odefunc, flux_0, times, rtol=r_tol, atol=a_tol)
        
                loss = torch.mean(torch.abs(pred_f - fluxes))
                losses.append(loss)
                print('Iter {:04d} | Total Loss {:.6f}'.format(itr, loss.item()))
                plt.plot(eval_times.tolist(), pred_interpolation.flatten().tolist())
                plt.show()
                ii += 1    

AssertionError: t must be strictly increasing or decrasing